# Rich Test

In [1]:
from ghtop.all_rich import pr,Panel,RenderGroup,Console,ConsoleOptions,Table,Live,Columns,Text,Style,ANSI_COLOR_NAMES,box
from ghtop.all_rich import Segment,RenderResult,Spinner,SpinnerColumn,BarColumn,Task,Progress,ProgressBar,Padding


import time,random
from collections import deque, OrderedDict
from typing import List, Dict
from fastcore.all import *
from ghapi.event import *
from collections import namedtuple

In [2]:
console = Console()

In [3]:
@delegates(Style)
def text(s, maxlen=None, **kwargs):
    "Create a styled `Text` object"
    if maxlen: s = truncstr(s, maxlen=maxlen)
    return Text(s, style=Style(**kwargs))

@delegates(Style)
def segment(s, maxlen=None, space=' ', **kwargs):
    "Create a styled `Segment` object"
    if maxlen: s = truncstr(s, maxlen=maxlen, space=space)
    return Segment(s, style=Style(**kwargs))

class Segments(list):
    def __init__(self, options): self.w = options.max_width
    
    @property
    def chars(self): return sum(o.cell_length for o in self)
    def txtlen(self, pct): return min((self.w-self.chars)*pct, 999)
    
    @delegates(segment)
    def add(self, x, maxlen=None, pct=None, **kwargs):
        if pct: maxlen = math.ceil(self.txtlen(pct))
        self.append(segment(x, maxlen=maxlen, **kwargs))

@delegates(Table)
def _grid(box=None, padding=0, collapse_padding=True, pad_edge=False, expand=False, show_header=False, show_edge=False, **kwargs):
    return Table(padding=padding, pad_edge=pad_edge, expand=expand, collapse_padding=collapse_padding,
                 box=box, show_header=show_header, show_edge=show_edge, **kwargs)

@delegates(_grid)
def grid(items, expand=True, no_wrap=True, **kwargs):
    g = _grid(expand=expand, **kwargs)
    for c in items[0]: g.add_column(no_wrap=no_wrap, justify='center')
    for i in items: g.add_row(*i)
    return g

Color = str_enum('Color', *ANSI_COLOR_NAMES)

In [4]:
evts = load_sample_events()

In [5]:
class Deque(deque):
    def __rich__(self): return RenderGroup(*(filter(None, self)))

In [6]:
@delegates()
class FixedPanel(Panel, GetAttr):
    _default='renderable'
    def __init__(self, height, **kwargs):
        super().__init__(Deque([' ']*height, maxlen=height), **kwargs)

    @delegates(Style)
    def add(self, s:str, **kwargs):
        "Add styled `s` to panel"
        self.append(text(s, **kwargs))

In [7]:
exs = [first(evts, risinstance(o)) for o in described_evts]

In [8]:
@patch
def __rich_console__(self:GhEvent, console, options):
    res = Segments(options)
    kw = {'color': colors[self.type]}
    res.add(self.emoji)
    res.add(self.actor.login, pct=0.25, bold=True, **kw)
    res.add(self.description, pct=0.5, **kw)
    res.add(self.repo.name, pct=0.5 if self.text else 1, space = ': ' if self.text else '', italic=True, **kw)
    if self.text: 
        clean_text = self.text.replace('\n', ' ').replace('\r', ' ')
        res.add (f'"{clean_text}"', pct=1, space='', **kw)
    res.add('\n')
    return res

In [9]:
colors = dict(
    PushEvent=None, CreateEvent=Color.red, IssueCommentEvent=Color.green, WatchEvent=Color.yellow,
    PullRequestEvent=Color.blue, PullRequestReviewEvent=Color.magenta, PullRequestReviewCommentEvent=Color.cyan,
    DeleteEvent=Color.bright_red, ForkEvent=Color.bright_green, IssuesEvent=Color.bright_magenta,
    ReleaseEvent=Color.bright_blue, MemberEvent=Color.bright_yellow, CommitCommentEvent=Color.bright_cyan,
    GollumEvent=Color.white, PublicEvent=Color.turquoise4)

colors2 = dict(
    PushEvent=None, CreateEvent=Color.dodger_blue1, IssueCommentEvent=Color.tan, WatchEvent=Color.steel_blue1,
    PullRequestEvent=Color.deep_pink1, PullRequestReviewEvent=Color.slate_blue1, PullRequestReviewCommentEvent=Color.tan,
    DeleteEvent=Color.light_pink1, ForkEvent=Color.orange1, IssuesEvent=Color.medium_violet_red,
    ReleaseEvent=Color.green1, MemberEvent=Color.orchid1, CommitCommentEvent=Color.tan,
    GollumEvent=Color.sea_green1, PublicEvent=Color.magenta2)

In [10]:
p = FixedPanel(15, box=box.HORIZONTALS, title='ghtop')
for e in evts[:163]: p.append(e)
p

In [11]:
p = FixedPanel(15, box=box.HORIZONTALS, title='ghtop')
for e in exs: p.append(e)
grid([[p,p]])

In [12]:
types = IssueCommentEvent,IssuesEvent,PullRequestEvent,PullRequestReviewEvent

In [13]:
ps = {o:FixedPanel(15, box=box.HORIZONTALS, title=camel2words(remove_suffix(o.__name__,'Event'))) for o in types}

In [14]:
for k,v in ps.items(): v.extend(evts.filter(risinstance(k)))
isc,iss,prs,prrs = ps.values()
grid([[isc,iss],[prs,prrs]], width=110)

## Animated Stats

This section outlines how we can display statistics and visualizations such as sparklines and status bars that are animated as events are received.

### `EProg` - Progress Bar

In [15]:
class EProg:
    "Progress bar with a heading `hdg`."
    def __init__(self, hdg='Quota', width=10):
        self.prog = Progress(BarColumn(bar_width=width), "[progress.percentage]{task.percentage:>3.0f}%")
        self.task = self.prog.add_task("",total=100, visible=False)
        store_attr()
    def update(self, completed): self.prog.update(self.task, completed=completed)
    def __rich_console__(self, console: Console, options: ConsoleOptions):
        self.prog.update(self.task, visible=True)
        yield grid([["Quota"], [self.prog.get_renderable()]], width=self.width+2, expand=False)


When you instantiate `Eprog` the starting progress is set to 0%:

In [16]:
p = EProg()
console.print(p)

Output()

You can update the progress bar with the `update` method:

In [17]:
p.update(10)
console.print(p)

### `Espark` - A sparkline combined with an EventTimer

In [18]:
class ESpark(EventTimer):
    "An `EventTimer` that displays a sparkline with a heading `nm`."
    def __init__(self, color, nm, store=5, span=.2, stacked=True): 
        super().__init__(store=store, span=span)
        store_attr()
    def _spark(self): return f"[{self.color}]{self.freq:.0f} {sparkline(self.hist)}[/]"
    def _nm(self): return f"[{self.color}]{self.nm}[/]"
    def __rich_console__(self, console: Console, options: ConsoleOptions): 
        yield grid([[self._nm()], [self._spark()]]) if self.stacked else f'{self._nm()}  {self._spark()}'

By default `nm` will be stacked on top of the sparkline like this:

In [19]:
e = ESpark(color='blue', nm='💌Issue')

def _r(): return random.randint(1,30)

def _sim(e, steps=8, sleep=.1):
    for i in range(steps):
        e.add(_r())
        time.sleep(sleep)

_sim(e)
console.print(e)

If you would prefer `nm` and the sparkline to be on one line instead, you can set `stacked` to `false`:

In [20]:
e = ESpark(color='blue', nm='💌Issue', stacked=False)
_sim(e)
console.print(e)

### SparkGrp - Sparklines, Progress bars and Statistics Combined

In [21]:
SparkCfg = namedtuple('Sparkcfg', ['color', 'nm'])

class SparkGrp:
    "Renders a group of `ESpark` along with a spinner and progress bar that are dynamically sized."
    def __init__(self, spks:List[SparkCfg], store=5, span=.2, stacked=True, max_width=console.width-5, spn:Spinner=Spinner('earth'), spn_lbl="/min"):
        self.sparklines = OrderedDict({s.nm: ESpark(color=s.color, nm=s.nm, store=store, span=span, stacked=stacked) for s in spks})
        self.slen = len(self.sparklines) * max(15, store*2)
        self.plen = max(max_width-self.slen-15, 15)
        self.progbar = EProg(width=self.plen)
        store_attr()
        
    def get_spk(self): 
        return grid([self.sparklines.values()], width=min(console.width-15, self.slen), expand=False)
    
    def get_spinner(self): return grid([[self.spn], [self.spn_lbl]])
    
    def update(self, events_dict:Dict[str,int], pct_complete:int=None):
        for k in events_dict: self.sparklines[k].add(events_dict[k])
        if pct_complete: self.progbar.update(pct_complete)
        
    def __rich_console__(self, console: Console, options: ConsoleOptions): 
        yield grid([[self.get_spinner(), self.get_spk(), grid([[self.progbar]], width=self.plen+5) ]], width=self.max_width)


Instantiate a `SparkGrp` by passing in a list of `SparkCfg`.  This builds a group of `ESpark` sparklines which you can display like so:

In [22]:
s = SparkGrp([SparkCfg('blue', '⭐Push'), 
              SparkCfg('red', '🎁Issue'), 
              SparkCfg('purple', '💌PRs'),
              SparkCfg('green', '🚀Release'), 
             ], stacked=True)

console.print(s)

Output()

You can add events to all `ESpark` instances of the `SparkGrp` at once by passing a dictionary to the `update` method.  Furthermore, you can pass the `pct_complete` argument to `update` to advance the progress bar:

In [23]:
def _d(): return {'⭐Push':_r(), '🎁Issue':_r(), '💌PRs':_r(), '🚀Release':_r()}

s.update(_d())
time.sleep(.1)
s.update(_d(), pct_complete=40)

console.print(s)

Here is what this looks like when animated using `Live`:

In [24]:
def _sim_spark(s, sleep=.05):
    with Live(refresh_per_second=20) as live:
        for i in range(101):
            evts = {k: _r() for k in s.sparklines.keys()}
            s.update(evts, pct_complete=i)
            live.update(s)
            time.sleep(sleep)
            
_sim_spark(s)

Output()

The progress bar will automatically resize depending on the number of sparklines you add to `SparkGrp`.  For example, if we only have one sparkline the width of objects dynamically change to look like this:

In [25]:
s = SparkGrp([SparkCfg('blue', 'foo')], store=20, stacked=True)
_sim_spark(s)

Output()

Output()

## Appendix

### Jeremy's Old Code

In [26]:
s = Spinner('earth')
def prog(o): return RenderGroup(ProgressBar(total=100, completed=o, width=8), f' {o/100:.0%}')
spk = sparkline([4,3,2,1])

def f(o):
    g = grid([[s, '⭐Push', '🎁[bright_magenta]Issue', '💌[blue]PRs[/]', '🚀[cyan]Release[/]', 'Quota'],
              ['/min', '89 ' + spk, f'[bright_magenta]38 {spk}[/]', f'[blue]19 {spk}[/]', f'[cyan]22 {spk}[/]', prog(o)]
             ], width=min(console.width,70))
    for c in g.columns[:-1]: c.title_justify = c.justify = 'center'
    g.columns[-1].width = 15
    return g

t = f(0)
t

In [27]:
with Live(refresh_per_second=20) as live:
    for o in range(1,15):
        live.update(f(o))
        time.sleep(0.1)

Output()